# Mathematics of Machine Learning
## Data Science for Data Scientists
---

# Mathematics & Method of Supervised Learning

This module helps lay the *conceptual* foundations of supervised machine learning. It is unlikely you will ever write code which looks exactly like this, you may, but more commonly you will rely on libraries which use these ideas and techniques.

It is important to understand the general appraoch to be able to use these libraries competently, and to understand the results they provide. 

For example LinearRegression is: empirical loss minimization with a square loss. Lasso regression is empirical loss minimization with a square regularization term. 

These terms should hopefully get clearer in the examples below, the key idea here is that "all this work is done for you". 

In [85]:
from sklearn.linear_model import LinearRegression, Lasso

## The Set Up of Supervised Machine Learning in terms of Loss 

a true relationship provides some true data

In [1]:
x_age = [18, 19, 20, 21]

# true relationship between age and height
def f(x):
    return 0.1*x + 0.01

y_true_height = [f(x) for x in x_age]

an estimate for the true relationship provides some estimate points

In [2]:
def fhat(x, a=0.1, b=0.05):
    return a*x + b


yhat = [fhat(x) for x in x_age]

In [3]:
print(yhat, y_true_height)

[1.85, 1.9500000000000002, 2.05, 2.15] [1.81, 1.9100000000000001, 2.01, 2.11]


we can define a loss *per point* which tells us how far away our estimates are from our predictions

In [4]:
def loss_point(prediction, observation):
    return abs(prediction - observation)

the total loss is the sum over all points

In [5]:
sum([loss_point(p, o) for p, o in zip(yhat, y_true_height)])

0.16000000000000014

since we are comparing our estimates to the true values, all errors are due to the *model* being inexact (rather than some "inherent" errors in the dataset).


## Loss Minimization

We rephrase the loss formula so it computes the total loss from the parameters direcly, ie., without having to precompute the predictions first.

In [6]:
def loss_fn(f_est, f_true, a, b):
    y = [f_true(x) for x in x_age]
    yhat = [f_est(x, a, b) for x in x_age]
    return sum([loss_point(p, o) for p, o in zip(yhat, y)])

With this formulation, you vary the parameters until the loss in minimal. 

In [7]:
loss_fn(fhat, f, 0.1, 0.01)

0.0

Suppose the true function were quadratic (ie., $ax^2 +bx + c$) then there would be a *necessary* true loss, due to using a poorly representative model. So "true loss" is a guide to how good the model is only, and does not arise from issues with the dataset. 

## Empirical Loss Minimization

In practice, we do not have $f$. 

We have dataset $\mathcal{D}$ which we *hope* "resembles" a true relationship $f$. The overall goal is to find an ideal, ie., *optimal*, estimate for f, $\hat{f}$, **using this dataset**.

How? Formulate a loss which describes how "close" $\hat{f}$ is to $\mathcal{D}$.

In [8]:
def loss_empirical(D, fhat, a, b):
    Dx, Dy = D
    yhat = [fhat(x, a, b) for x in Dx]
    return sum([loss_point(p, o) for p, o in zip(yhat, Dy)])

We assume $\mathcal{D}$ is generated from a true model, just with some random error...

In [9]:
from random import random as rerror

In [10]:
Dx_school = (2, 3, 4)
Dy_school = [ 2*x + 1 + rerror() for x in Dx_school ]

Here the true model is $f(x) = 2x + 1$ .

In practice we wouldnt know this, we would just have $\mathcal{D}$ as below.

In [12]:
D_school = [Dx_school, Dy_school]

In [13]:
D_school

[(2, 3, 4), [5.204371643320275, 7.614282900476174, 9.732090790868954]]

Our model, $\hat{g}$ will be linear, ...

In [14]:
def ghat(x, a, b):
    return a*x + b

Even if we use this model with the "true" parameters, the *empirical loss* still shows an error. Now due to the dataset cotaining "random" variation. 

In [15]:
loss_empirical(D_school, ghat, 2, 1)

1.550745334665403

The danger here is that we optimize for the dataset and thereby arrive at parameters very different from the truth.

This is unlikely to be serious in the cases where linear models, etc. direclty & obviously apply. Issue is more common in complex datasets. 

## How do you choose the best parameters?

Insight: the rate of change of the total loss is a guide to how fast to change the parameters.

### Direction

eg., suppose you move too far in the wrong direction

In [16]:
loss_empirical(D_school, ghat, 2, 1.5)

0.6420020480248523

In [17]:
loss_empirical(D_school, ghat, 2, 0.5)

3.050745334665403

You went too far, ie., lowered b too much. Perhaps stop at your best previous $b$.

### Pace

Big changes to the loss suggest you should make big changes to the parameters, ie., use some percetage of the total loss as *how much* you're going to change the total parameter.

Below, we update $b$ using 10% of the total loss of the previous step.

Start at some random initial values for $a, b$...

In [18]:
loss_empirical(D_school, ghat, 2, 10)

25.4492546653346

Choose a parameter to optimize, say $b$, and move by 10%...

In [19]:
loss_empirical(D_school, ghat, 2, 10 - 2.6)

17.6492546653346

In [20]:
loss_empirical(D_school, ghat, 2, 10 - 2.6 - 1.8 - 1.2 - 0.93) # 5 steps in one line

5.859254665334599

In [21]:
loss_empirical(D_school, ghat, 2, 10 - 2.6 - 1.8 - 1.2 - 0.93 - 0.65 - 0.45 - 0.32 - 0.26 - 0.148 - 0.1 - 0.08 - 0.08 - 0.07 - 0.07)

0.9000020480248514

At some point the loss changes direction, ie., increases again.

In [22]:
loss_empirical(D_school, ghat, 2, 10 - 2.6 - 1.8 - 1.2 - 0.93 - 0.65 - 0.45 - 0.32 - 0.26 - 0.148 - 0.1 - 0.08 - 0.08 - 0.07 - 0.07 - 0.06)

1.0047453346654018

Here, we stop.

In [23]:
b_optimal = 10 - 2.6 - 1.8 - 1.2 - 0.93 - 0.65 - 0.45 - 0.32 - 0.26 - 0.148 - 0.1 - 0.08 - 0.08 - 0.07 - 0.07

In [24]:
b_optimal

1.242

In mathematical notation optimial values are sometimes written with a star, here: $b^*$.

NB. We cheated by starting with the optimal $a$ and just optimizing $b$... to do this properly we need a more sophisticated algoithm

## Exercise

Convince yourself you understand the set up and by-hand process of loss minimization.

* define a true function `h`, eg., 2x + 3
* define a estimate called `hhat`, 2.1x + 2.9
* define a dataset for `x`, eg., `list(range(0, 10))`
* generate a dataset for `y` and for `yhat`
    * either loop over `x` and `.append` a `h(x)` to a `y` list
    * or use a comprehension as above
    
* define a *square loss* per-point which is `(pred - obs) ** 2`
* compute the true total loss 
    * run your per-point loss over all `y` and `yhat`
    * `sum()`
    
* by-hand, update your `hhat` function to have different `a, b` values
    * recompute the total loss
    * update your `hhat` until the total loss is minimal
        * you should be able to get `0` total loss, because you are comparing with the true function
        
* Stretch Exercise:
    * as-above, with a dataset
    * hint: copy/paste my `loss_empirical` function & generate your own dataset as I have

## Remarks about the Loss Function

There is no "best" formula for the loss... it is just any formula which tracks "how well" $\hat{f}$ estimates the domain of interest. 

Some considerations: some loss formulae are more useful for by-hand mathematics or certain algorithms. Here the *square-loss* is *smooth* so its rate of change is continuous, ie., it will always change without gaps. Meaning it is "safe and easy" to use this rate of change of this loss to update my parameters. 

Aside: in practice, the absolute loss tends to perform better ie., provide better parameter estiamtes.

## Regularization

The loss optimization processs is *empirical* ie., relative to a dataset and therefore cannot be "trusted" to yield the true result. 

The question for regularization is: Can we change the empirical loss formula to help?

In [79]:
def loss_empirical_old(D, fhat, a, b):
    Dx, Dy = D
    yhat = [fhat(x, a, b) for x in Dx]
    return sum([(p - o)**2 for p, o in zip(yhat, Dy)])

def loss_empirical_reg(D, fhat, a, b):
    Dx, Dy = D
    yhat = [fhat(x, a, b) for x in Dx]
    return sum([(p - o)**2 + 0.15*(a + b)**2 for p, o in zip(yhat, Dy)])

The above modification produces a *lower* loss for the true value of $b$,...

In [80]:
loss_empirical_reg(D_school, ghat, 2, 1.242)

5.109953234507925

In [81]:
loss_empirical_reg(D_school, ghat, 2, 1)

5.005068176485981

ie., we would now prefer $b^* = 1$

(Still, it is unlikely to settle on `1`...)

In [82]:
loss_empirical_reg(D_school, ghat, 2, 1.05)

4.993618643019439

### How does this work?

We added a term to the loss which had *nothing to do* with the fit to dataset; ie., a term involving $a, b$ only.

$a, b$ are the heart of the model, if you optimize for *something* to do with only $a, b$ then you impose some principles on the model itself: eg., that it should be simple, etc.

Suppose there is some dataset error, eg., `2.5` adding the total of $(a + b)^2$  forces the error to increase. By forcing the model to consider both equally, it drags the model away from extreme values of $a, b$ which tends to be caused by misfitting. 

For the same dataset error, higher values for both parameters is penalized..

In [71]:
2.5 + (2 + 1.1) ** 2

12.110000000000001

In [72]:
2.5 + (2 + 1.2) ** 2

12.740000000000002

In [78]:
2.5 + (2.1 + 1.1) ** 2

12.740000000000002

How much to penalize these high-values is a hyper-parameter, ie., up to you to decide. Below, 15% of the term is used..

In [75]:
2.5 + 0.15*(2 + 1.1) ** 2

3.9415000000000004

In [74]:
2.5 + 0.15*(2 + 1.2) ** 2

4.0360000000000005

NB. recall, given the dataset error is fixed, we will chose a *lower* $b$  because the loss is lower. Here then, we would prefer $1.1$.

In practice, by choosing $1.1$ we may increase the dataset error: so there is a tug-of-war between dataset fit and "balancing parameters". The *strength* of the regularization, ie., how much simplicity wins is given by the 15% above.  At 0% the dataset determines everything.

If the dataset error increases at 1.1, and we use only 15% of the regularization term, we still may prefer the higher $b$...

In [76]:
2.6 + 0.15*(2 + 1.1) ** 2

4.0415

In [77]:
2.4 + 0.15*(2 + 1.2) ** 2

3.936

### Aside: interpreting (a + b)^2

Recall, $(a + b)^2 = a^2 + b^2 + 2ab$

So the loss formula is, $l(a, b; x, y) = error_{data} + error_{reg} = (ax + b - y)^2 +  a^2 + b^2 + 2ab$

Think about optimizing this formula, ie., finding its lowest value by choosing $a, b$. 

The ideal value for each term is $0$ or as low as possible. 

Forces driving the choice of $a,b$...

1. $(ax + b - y)^2$ is zero/low when $ax + b = y$, so $y$ drives $a, b$ up "to meet it".
2. $a^2$ is low when $a$ is low. 
3. $b^2$ is low when $b$ is low.
4. $2ab$ is low when *both* $a$ and $b$ is low!

Force `1` prefers $a, b$ to meet $y$. Forces `2, 3` prefer the lowest $a,b$ possible; and force `4` is happy only when  both $a, b$ is low.

So $error_{reg}$ causes lowering $a, b$  individually to be optimial, and lowering *both at the same time*. This corresponds to models which don't "put all their eggs in one basket", ie., happen to get a good fit because of a good slope. It requires models to fit the data, "and then" try to balance that fit by both learning a non-extreme (ie., low) slope and an intercept. 


